<a href="https://colab.research.google.com/github/hank199599/deep_learning_keras_log/blob/main/Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 超越序列式模型

* 單輸入模型
* 多輸入模型
* 殘差連接層(residual connections)層：透過特徵圖(張量)相加，將先前的資訊重新注入下游資料流

## [函數式API](https://keras.io/guides/functional_api/)

使用Input()方法來檢立一個張量，並將張量直接傳入層(layer)或模型(model)之中，  
取得處理後的結果張量。
```python
from keras import Input,layers

input_tensor = Input(shape=(32,)) #建立一個輸入張量
print(input_tensor.shape)

dense = layers.Dense(16,activation='relu') #建立一個Dense層，並將其想像成一個函數
output_tensor = dense(input_tensor) # 將張量輸入層函數，他會回傳經處理後的結果張亮
print(output_tensor.shape)
```

### 序列式 與 函數式 

序列式(Sequential)建立模型  
![pic 7-1](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/7-1.png)

In [1]:
from keras.models import Sequential,Model
from keras import layers,Input

model=Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(32,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
Total params: 4,192
Trainable params: 4,192
Non-trainable params: 0
_________________________________________________________________


#### 函數式(API)建立模型
透過建立**Model物件**，


In [2]:
input_tensor = Input(shape=(64,)) # 建立一個初始張量

x = layers.Dense(32,activation='relu')(input_tensor)

y = layers.Dense(32,activation='relu')(x)

output_tensor = layers.Dense(10,activation='softmax')(y)

model = Model(input_tensor,output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


如果用完全不相干的輸入和輸出張量去建構模型。  
因為Keras找不到相關資訊，導致執行時會生錯誤。

In [3]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input,output_tensor)

ValueError: ignored

在編譯、訓練或驗證此Model物件時，API的功能與序列式模型相同

In [4]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
import numpy as np

x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

# 將訓練輸入模型進行訓練
model.fit(x_train,y_train,epochs=10,batch_size=128)
score = model.evaluate(x_train,y_train)
print(score) 


Epoch 1/10
8/8 [==============================] - 1s 2ms/step - loss: 12.2086
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.1521
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 12.6796
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 13.6005
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 14.9342
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 16.7674
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 18.6711
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 21.3209
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 24.0460
Epoch 10/10
32/32 [==============================] - 0s 931us/step - loss: 30.1749
30.174882888793945


## 多輸入模型

*[範例]*：典型的問答模型  
必須針對問題產生出答案，可透過softmax分類器針對某些事件先定義好詞彙並輸出答案  
![pic 7-3](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/7-3.png)


In [5]:
from keras import Model 
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,),dtype='int32',name='text')
embedded_text = layers.Embedding(text_vocabulary_size,64)(text_input)
print(embedded_text.shape)
encoded_text = layers.LSTM(32)(embedded_text)
print(encoded_text.shape)

question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
print(embedded_question.shape)
encoded_question = layers.LSTM(16)(embedded_question)
print(encoded_question.shape)

concatenated = layers.concatenate([encoded_question,encoded_text],axis=1)
print(concatenated.shape)

answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)
print(answer.shape)

model = Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

(None, None, 64)
(None, 32)
(None, None, 32)
(None, 16)
(None, 48)
(None, 500)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      text[0][0]                       
_____________

### 訓練雙輸入模型的方法
1. 為模型準備Numpy陣列
2. 選擇下列兩種方式進行訓練 
①將 Numpy 陣列資料組成串列(list)做為輸入進行訓練。  
②建立一個字典(dict), 將輸入透過鍵(輸入名稱),對應到值(Numpy陣列資料)。當然,此方法只有在為輸入命名時才可用,例如上面程式中的第7行
```python
Input(., name = 'text,
```
我們就可以建立 dict:
```python
{'text': Numpy 資料}  
```
做為此輸入張量對應的 Numpy 資料。

In [7]:
import numpy as np

num_samples = 1000
max_length = 100

# 產生虛擬text資料：1000筆，每筆100個字(數字)
text = np.random.randint(1,text_vocabulary_size,size = (num_samples,max_length))
print(text.shape)

# 產生虛擬question資料：1000筆，每筆100個字(數字)
question = np.random.randint(1,question_vocabulary_size,size = (num_samples,max_length))
print(question.shape)

# 產生虛擬answer資料：1000筆，每筆100個字(數字)
answers = np.zeros(shape=(num_samples,answer_vocabulary_size),dtype='int32')

for answer in answers:
  answer[np.random.randint(answer_vocabulary_size)]=1
print(answers.shape)

# 訓練方法1：使用list將送入資料進行訓練
model.fit([text,question],answers,epochs=10,batch_size=128)

# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'text':text,'question':question},answers,epochs=10,batch_size=128)


(1000, 100)
(1000, 100)
(1000, 500)
Epoch 1/10
8/8 [==============================] - 1s 128ms/step - loss: 6.2561 - acc: 0.0030
Epoch 2/10
8/8 [==============================] - 1s 126ms/step - loss: 6.1804 - acc: 0.0030
Epoch 3/10
8/8 [==============================] - 1s 112ms/step - loss: 6.1194 - acc: 0.0090
Epoch 4/10
8/8 [==============================] - 1s 137ms/step - loss: 6.0011 - acc: 0.0080
Epoch 5/10
8/8 [==============================] - 1s 115ms/step - loss: 5.8977 - acc: 0.0090
Epoch 6/10
8/8 [==============================] - 1s 115ms/step - loss: 5.7931 - acc: 0.0180
Epoch 7/10
8/8 [==============================] - 1s 114ms/step - loss: 5.7211 - acc: 0.0270
Epoch 8/10
8/8 [==============================] - 1s 126ms/step - loss: 5.6355 - acc: 0.0390
Epoch 9/10
8/8 [==============================] - 1s 116ms/step - loss: 5.5607 - acc: 0.0460
Epoch 10/10
8/8 [==============================] - 1s 113ms/step - loss: 5.4866 - acc: 0.0350
Epoch 1/10
8/8 [=================

## 多輸出模型
使用函數式api建構多個輸出的模型  
在這情形下需要依據不同的輸出指定不同的損失函數來計算損失值  
**由於梯度下降要求純量最小化，因此必須將這些損失值結合成單一數值才能訓練模型**
  

**例子**：以一個神經網路同時預測資料中的不同屬性  
![pic 7-4](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/7-4.png)

In [9]:
from keras import layers,Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10 #將收入分成10群

posts_input = Input(shape=(None,),dtype='int32',name='posts')

# 用函數式API將輸入向量傳入Embedding層，得到維度為256的崁入向量
embedding_posts = layers.Embedding(vocabulary_size,256)(posts_input)
print(embedding_posts.shape)

# 以函數式API將砍入向量傳入一層層之中處理
x = layers.Conv1D(128,5,activation='relu')(embedding_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128,activation='relu')(x)
print(x.shape) # 走過一連串層後，x.shape為(?,128)

#======================================================================#
#將x向量分別送到3個輸入層
#======================================================================#
# 1. 預測年紀的輸出層：純量迴歸任務
age_prediction = layers.Dense(1,name='age')(x)

# 2. 預測收入族群的輸出層：多分類任務
income_prediction = layers.Dense(num_income_groups,activation='softmax',name='income')(x)

# 3. 預測性別的輸出層：二元分類任務
gender_prediction = layers.Dense(1,activation='softmax',name='gender')(x)

#======================================================================#
#用輸入向量與輸出向量實例化Model物件
#======================================================================#
model = Model(posts_input,[age_prediction,income_prediction,gender_prediction])

model.summary()

(None, None, 256)
(None, 128)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, None, 128)    163968      embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, None, 128)    0           conv1d_5[0][0]                   
______________________________________________________________

#### 計算整體誤差的方式
在編譯模型時指定計算整體誤差的方式

1. 使用損失串列 (loss list)：需照層的建立順序排序



In [ ]:
model.comile(optimizer='rmsprop',loss=['mse','categorical_crossentropy','binary_crossentropy'])

2. 使用損失字典 (loss dict)：需為輸出層指定名稱

In [ ]:
model.comile(optimizer='rmsprop',loss={'age':'mse','income':'categorical_crossentropy','gender':'binary_crossentropy'})

若模型間，有十分不平衡的損失值  
易導致模型優先針對損失值較大者進行優化而忽略其他模型  
我們可以在compile()加入**loss_weight**參數為損失值指定不同程度的重要性

##### loss_weight參數設定
* 均方誤差 (MAE)：3-5
* 交叉熵 (cross-entropy):0.1

1. 使用損失串列 (loss list)：需照層的建立順序排序



In [ ]:
model.comile(optimizer='rmsprop',loss=['mse','categorical_crossentropy','binary_crossentropy'],loss_weights=[0.25,1.,10.])

2. 使用損失字典 (loss dict)：需為輸出層指定名稱

In [ ]:
model.comile(optimizer='rmsprop',loss={'age':'mse','income':'categorical_crossentropy','gender':'binary_crossentropy'},loss_weights={'age':0.25,'income':1,'gender':10.})

#### 訓練整體誤差的方式
與多輸入模型的訓練方式一樣，可透過串列或字典將Numpy資料傳遞給模型進行訓練

1. 使用損失串列 (loss list)：需照層的建立順序排序



In [ ]:
model.fir(posts,[age_targets,income_targets,gender_targets],epochs=10,batch_size=64)

2. 使用損失字典 (loss dict)：需為輸出層指定名稱

In [ ]:
model.fir(posts,{'age':age_targets,'income':income_targets,'gender':gender_targets},epochs=10,batch_size=64)